In [25]:
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
import pandas as pd


In [26]:
model = tf.keras.models.load_model('lstm_stock_model.h5')
stock_symbol="AAPL"

In [27]:
def predict_next_day(stock_symbol: str, model):
    
    # Preprocess the data
    X, y, scaler = preprocess_data(stock_symbol)

    # Make predictions
    y_pred = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred)
    y_test_actual = scaler.inverse_transform(y.reshape(-1, 1))
    
    return {"predicted_price": round(float(y_pred[-1]), 2)}


def preprocess_data(stock_symbol):
    start_date = '2018-01-01'
    end_date = datetime.now()
    
    data = yf.download(stock_symbol, start=start_date, end=end_date)
            
    data = data[['Adj Close']].rename(columns={'Adj Close': 'Close'})
    scaler = MinMaxScaler(feature_range=(0, 1))
    data['Close'] = scaler.fit_transform(data[['Close']])
    data_values = data.values
    
    x, y = create_sequences(data_values)
    x = x.reshape((x.shape[0], x.shape[1], 1))
    
    return x, y, scaler


def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)


def evaluate_models_performance_last_30_days(stock_symbol: str, model):
   # Preprocess the data
    X, y, scaler = preprocess_data(stock_symbol)

    # Make predictions
    y_pred = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred)
    y_test_actual = scaler.inverse_transform(y.reshape(-1, 1))
    
    
    
    mae = mean_absolute_error(y_test_actual, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))
    
    return mae, rmse, y_pred

In [28]:
predict_next_day(stock_symbol=stock_symbol, model=model)

[*********************100%***********************]  1 of 1 completed


53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


/tmp/ipykernel_51932/538459962.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"predicted_price": round(float(y_pred[-1]), 2)}


{'predicted_price': 215.62}

In [19]:
mae, rmse, y_pred = evaluate_models_performance_last_30_days(stock_symbol=stock_symbol, model=model)

[*********************100%***********************]  1 of 1 completed

18/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [41]:
x

array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [38]:

price_list = pd.DataFrame([100.5] * 61, columns=['Close'])
scaler = MinMaxScaler(feature_range=(0, 1))
price_list = scaler.fit_transform(price_list[['Close']])
x, y = create_sequences(price_list)
x = x.reshape((x.shape[0], x.shape[1], 1))

y_pred = model.predict(price_list)

y_pred = scaler.inverse_transform(y_pred)

y_pred[-1]



ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [45]:
# Convert the list to a pandas DataFrame for consistency
new_data = [150.0, 155.0, 160.0, 158.0, 159.0, 165.0, 170.0, 175.0, 180.0, 185.0,
            190.0, 195.0, 200.0, 205.0, 210.0, 215.0, 220.0, 225.0, 230.0, 235.0,
            240.0, 245.0, 250.0, 255.0, 260.0, 265.0, 270.0, 275.0, 280.0, 285.0,
            290.0, 295.0, 300.0, 305.0, 310.0, 315.0, 320.0, 325.0, 330.0, 335.0,
            340.0, 345.0, 350.0, 355.0, 360.0, 365.0, 370.0, 375.0, 380.0, 385.0,
            390.0, 395.0, 400.0, 405.0, 410.0, 415.0, 420.0, 425.0, 430.0, 435.0,
            440.0, 445.0, 450.0, 455.0, 460.0, 465.0, 470.0, 475.0]

new_data_df = pd.DataFrame(new_data, columns=['Close'])

# Normalize the new data using the same scaler (must have been fitted on the training data)
scaler = MinMaxScaler(feature_range=(0, 1))
new_data_scaled = scaler.fit_transform(new_data_df[['Close']])

# We want to use the last 60 prices for prediction
sequence = new_data_scaled[-60:].reshape((1, 60, 1))  # Reshape to (1, 60, 1) to match LSTM input format

# Make a prediction using the model
predicted_price = model.predict(sequence)

# The predicted price (next day's price) after unscaling
predicted_price_unscaled = scaler.inverse_transform(predicted_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


In [46]:
predicted_price_unscaled

array([[429.32562]], dtype=float32)